In [2]:
import pandas as pd
import pickle
import sympy
import networkx as nx
import numpy as onp
import matplotlib.pyplot as plt

In [3]:

#data=pd.read_csv('complex toy.csv')
#with open("complex toy graph.p", 'rb') as f:  # notice the r instead of w
#    true_graph = pickle.load(f)
#X=data.to_numpy(dtype=onp.float32)[:1000].T

In [4]:
#plt.figure(figsize=(15,15))
#plt.margins(x=0.4)
#nx.draw(true_graph,with_labels=True,labels=nx.get_node_attributes(true_graph,'equation'),font_size=20)

In [5]:
#ga=X[4]
#X_used=onp.delete(X,[4,7],axis=0)

In [6]:
X=10*onp.random.rand(4,1000)
Y1=X[0]*X[1]
Y2=X[1]+onp.sin(X[3])
ga=Y2
X_used=onp.concatenate([X,onp.expand_dims(Y1,axis=0)])
X=onp.concatenate([X,onp.expand_dims(Y1,axis=0),onp.expand_dims(Y2,axis=0)])
which=onp.array([0,1,2,3,4])

In [7]:
beta=onp.array([0.1,0.01,0.001])

In [12]:
constant_mat=onp.ones((X.shape[1],X.shape[1]))
linear_mat=onp.expand_dims(X,-1)*onp.expand_dims(X,1)
quadratic_mat=onp.expand_dims(linear_mat,0)*onp.expand_dims(linear_mat,1)
diff_X=onp.tile(onp.expand_dims(X,-1),(1,1,X.shape[1]))-onp.tile(onp.expand_dims(X,1),(1,X.shape[1],1))
gaussian_mat=onp.exp(-(diff_X)**2/2)
K=constant_mat+beta[0]*onp.sum(linear_mat,axis=0)+beta[1]*onp.sum(quadratic_mat,axis=(0,1))+beta[2]*onp.prod(gaussian_mat+onp.ones_like(gaussian_mat),axis=0)



eigenvalues,alphas = onp.linalg.eigh(K)
max_eigenval=eigenvalues[-1]

norms=onp.sqrt(onp.sum(alphas**2,axis=0))
alphas=norms[onp.newaxis,...]*alphas
alphas=alphas[eigenvalues/max_eigenval>1e-6]


constant_mat=onp.ones((X_used.shape[1],X_used.shape[1]))
linear_mat=onp.expand_dims(X_used,-1)*onp.expand_dims(X_used,1)
quadratic_mat=onp.expand_dims(linear_mat,0)*onp.expand_dims(linear_mat,1)
diff_X=onp.tile(onp.expand_dims(X_used,-1),(1,1,X_used.shape[1]))-onp.tile(onp.expand_dims(X_used,1),(1,X_used.shape[1],1))
gaussian_mat=onp.exp(-(diff_X)**2/2)
K=constant_mat+beta[0]*onp.sum(linear_mat,axis=0)+beta[1]*onp.sum(quadratic_mat,axis=(0,1))+beta[2]*onp.prod(gaussian_mat+onp.ones_like(gaussian_mat),axis=0)

alphas.shape

(19, 1000)

In [11]:
alphas@onp.ones((1000,1000))@alphas.T

array([[ 1.34451372e+00, -2.08818168e+00, -1.13873948e+00,
         1.20332599e-01, -1.83838814e-01,  6.05507181e-01,
         2.77037951e-01,  1.25279500e+00,  1.99134194e-01,
        -2.07073119e-01, -2.23120352e+00, -1.81402103e+00,
        -7.46527560e-01, -2.35030955e-01, -6.18273379e-02,
         4.72205744e-02,  1.38513898e+00,  2.03935338e+00,
         1.06805174e+00],
       [-2.08818168e+00,  3.24318204e+00,  1.76859105e+00,
        -1.86890120e-01,  2.85522446e-01, -9.40421053e-01,
        -4.30271233e-01, -1.94573216e+00, -3.09277899e-01,
         3.21607945e-01,  3.46531111e+00,  2.81737957e+00,
         1.15944163e+00,  3.65029624e-01,  9.60248400e-02,
        -7.33388863e-02, -2.15127731e+00, -3.16734617e+00,
        -1.65880500e+00],
       [-1.13873948e+00,  1.76859105e+00,  9.64458441e-01,
        -1.01916016e-01,  1.55702774e-01, -5.12835923e-01,
        -2.34638032e-01, -1.06105807e+00, -1.68657237e-01,
         1.75381131e-01,  1.88972377e+00,  1.53638995e+00,
    

In [8]:
#from sklearn.gaussian_process.kernels import RBF
#kernel = 1.0 * RBF(1.0)
#gaussian_mat=onp.stack([kernel(X_used[i:i+1].T) for i in range(7)])

In [14]:
gamma=1e-15

In [15]:
new_ga=alphas@ga
new_K=alphas@(K+gamma*onp.eye(K.shape[0]))@alphas.T
new_K.shape

(19, 19)

# all levels

In [16]:

yb=-onp.linalg.solve(new_K,new_ga)
print(onp.dot(yb,yb),onp.dot(new_ga,yb),onp.dot(yb,new_K@yb))

noise=-gamma*onp.dot(yb,yb)/onp.dot(new_ga,yb)
noise

294.3117210813561 -18.486287626345955 18.486287625975436


1.5920542135345456e-14

# linear

In [17]:
Kl=beta[0]*onp.sum(linear_mat,axis=0)+constant_mat
Kl=alphas@(Kl)@alphas.T
eigenvalues_linear,alphas_linear = onp.linalg.eigh(Kl)
noise_level_linear=gamma+abs(eigenvalues_linear[0])-eigenvalues_linear[0]

#norms=onp.sqrt(onp.sum(alphas_linear**2,axis=0))
#alphas_linear=norms[onp.newaxis,...]*alphas_linear
#alphas_linear=alphas_linear[eigenvalues_linear/max_eigenval>1e-4]


ybl=-onp.linalg.solve(Kl+noise_level_linear*onp.eye(Kl.shape[0]),new_ga)
#print(Kl)
#print(ybl)
print('gamma',gamma)
print('smallest eigenvalue of Kl',onp.linalg.eigh(Kl)[0][0])
print('noise level',noise_level_linear)
#print(onp.dot(ga_linear,ybl))
linear_noise=-noise_level_linear*onp.dot(ybl,ybl)/onp.dot(new_ga,ybl)
linear_noise

gamma 1e-15
smallest eigenvalue of Kl -4.650678049036785e-13
noise level 9.31135609807357e-13


1.0550624200251841

In [18]:
Kl=beta[0]*onp.sum(linear_mat,axis=0)+constant_mat

gamma=1e-15
eigenvalues_linear,_ = onp.linalg.eigh(Kl)
noise_level_linear=gamma+abs(eigenvalues_linear[0])-eigenvalues_linear[0]
print(noise_level_linear)
ga_linear=ga
Kl=Kl+noise_level_linear*onp.eye(Kl.shape[0])

ybl=-onp.linalg.solve(Kl,ga_linear)
#print(Kl)
#print(ybl)
#print(onp.dot(ga_linear,ybl))
linear_noise=-noise_level_linear*onp.dot(ybl,ybl)/onp.dot(ga_linear,ybl)
linear_noise

8.543096584822005e-11


0.9992022425628522

In [19]:
Kl.shape

(19, 19)

# quadratic

In [84]:
Kq=constant_mat+beta[0]*onp.sum(linear_mat,axis=0)+beta[1]*onp.sum(quadratic_mat,axis=(0,1))
Kq=beta[1]*onp.sum(quadratic_mat,axis=(0,1))

eigenvalues_quadratic,alphas_quadratic = onp.linalg.eigh(Kq+gamma*onp.eye(Kq.shape[0]))
max_eigenval=eigenvalues_quadratic[-1]
noise_level_quadratic=gamma+abs(eigenvalues_quadratic[0])-eigenvalues_quadratic[0]

norms=onp.sqrt(onp.sum(alphas_quadratic**2,axis=0))
alphas_quadratic=norms[onp.newaxis,...]*alphas_quadratic
alphas_quadratic=alphas_quadratic[eigenvalues_quadratic/max_eigenval>1e-4]
alphas_quadratic=onp.eye(ga.shape[0])
ga_quadratic=alphas_quadratic@ga
Kq=alphas_quadratic@(Kq+noise_level_quadratic*onp.eye(Kq.shape[0]))@alphas_quadratic.T


ybq=-onp.linalg.solve(Kq,ga_quadratic)
quadratic_noise=-noise_level_quadratic*onp.dot(ybq,ybq)/onp.dot(ga_quadratic,ybq)
quadratic_noise

0.9982749958881375

Thus all levels are needed
# computing activations

In [86]:
def all_but_i(i,linear_mat,quadratic_mat,non_linear_mat):
    res=onp.zeros_like(linear_mat[0])
    res+=beta[0]*(onp.sum(linear_mat[:i],axis=0)+onp.sum(linear_mat[i+1:],axis=0))
    #print('linear\n',res)
    res+=beta[1]*onp.sum(quadratic_mat[:i,:i],axis=(0,1))
    res+=beta[1]*onp.sum(quadratic_mat[:i,i+1:],axis=(0,1))
    res+=beta[1]*onp.sum(quadratic_mat[i+1:,:i],axis=(0,1))
    res+=beta[1]*onp.sum(quadratic_mat[i+1:,i+1:],axis=(0,1))
    #print('quadratic\n',res)
    new_non_linear_mat=onp.delete(non_linear_mat,i,axis=0)
    plus_one=new_non_linear_mat+onp.ones_like(new_non_linear_mat)
    res+=beta[2]*onp.prod(plus_one,axis=0)
    #print('non lineaar\n',res)
    return res+onp.ones_like(res)

def only_i(i,linear_mat,quadratic_mat,non_linear_mat):
    res=onp.zeros_like(linear_mat[0])
    res+=beta[0]*linear_mat[i]
    res+=beta[1]*onp.sum(quadratic_mat[i],axis=0)
    new_non_linear_mat=onp.delete(non_linear_mat,i,axis=0)
    res+=beta[2]*non_linear_mat[i]*onp.prod(new_non_linear_mat+onp.ones_like(new_non_linear_mat),axis=0)
    return res+onp.ones_like(res)

def activation_of_i(i,yb,noise,ga,alphas,linear_mat,quadratic_mat,gaussian_mat):
    Ki=alphas@all_but_i(i,linear_mat,quadratic_mat,gaussian_mat)@alphas.T
    #print(onp.linalg.norm(Ki))
    return 1-noise+onp.dot(yb,Ki@yb)/onp.dot(yb,ga)

def activation_of_i_2(i,yb,noise,ga,alphas,linear_mat,quadratic_mat,gaussian_mat):
    Ki=alphas@only_i(i,linear_mat,quadratic_mat,gaussian_mat)@alphas.T
    #print(onp.linalg.norm(Ki))
    return -onp.dot(yb,Ki@yb)/onp.dot(yb,ga)

In [96]:
def prune_ancestors(which,yb,noise,linear_mat,quadratic_mat,non_linear_mat,new_ga,alphas):
    activations=[activation_of_i(i,yb,noise,new_ga,alphas,linear_mat,quadratic_mat,non_linear_mat) for i in range(which.shape[0])]
    print('noise',noise)
    print('activation 1',onp.stack([which,activations]))
    least_activated=onp.argsort(activations)[0]
    new_which=onp.delete(which,least_activated,axis=0)
    
    new_linear_mat=onp.delete(linear_mat,least_activated,axis=0)
    new_quadratic_mat=onp.delete(quadratic_mat,least_activated,axis=0)
    new_quadratic_mat=onp.delete(new_quadratic_mat,least_activated,axis=1)
    new_non_linear_mat=onp.delete(non_linear_mat,least_activated,axis=0)
    K=beta[0]*onp.sum(new_linear_mat,axis=0)+beta[1]*onp.sum(new_quadratic_mat,axis=(0,1))+beta[2]*onp.prod(new_non_linear_mat+onp.ones_like(new_non_linear_mat),axis=0)
    K+=onp.ones_like(K)+gamma*onp.eye(K.shape[0])
    K=alphas@K@alphas.T
    print('min and max eigenvalues',onp.delete(onp.linalg.eigvalsh(K),[k for k in range(1,K.shape[0]-1)]))
    new_yb=-onp.linalg.solve(K,new_ga)
    new_noise=-gamma*onp.dot(new_yb,new_yb)/onp.dot(new_ga,new_yb)
    print('new noise',new_noise)
    if new_noise>0.5:
        return yb,which,linear_mat,quadratic_mat,non_linear_mat
    else:
        if len(activations)==2:
            return new_yb,new_which,new_linear_mat,new_quadratic_mat,new_non_linear_mat
        return prune_ancestors(new_which,new_yb,new_noise,new_linear_mat,new_quadratic_mat,new_non_linear_mat,new_ga,alphas)


In [97]:
new_yb,new_which,new_linear_mat,new_quadratic_mat,new_non_linear_mat=prune_ancestors(which,yb,noise,linear_mat,quadratic_mat,gaussian_mat,new_ga,alphas)
new_which

noise 1.1787525909477545e-14
activation 1 [[0.         1.         2.         3.         4.        ]
 [0.22559788 0.23421854 0.22652888 0.96394167 0.12104288]]
min and max eigenvalues [6.63787588e-05 1.60317957e+05]
new noise 4.824653777594712e-14
noise 4.824653777594712e-14
activation 1 [[0.         1.         2.         3.        ]
 [0.20128774 0.20771248 0.20213436 0.98691861]]
min and max eigenvalues [2.97265831e-08 1.02344933e+05]
new noise 3.95480454072759e-12
noise 3.95480454072759e-12
activation 1 [[1.         2.         3.        ]
 [0.18808241 0.18238502 0.99494052]]
min and max eigenvalues [-8.03697397e-12  5.71905386e+04]
new noise 0.08328395347193181
noise 0.08328395347193181
activation 1 [[1.         3.        ]
 [0.04425121 0.31933961]]
min and max eigenvalues [-4.96236061e-12  2.26725520e+04]
new noise 11.002396818527481


array([1, 3])